### In this version, the nonattending feature has only 1 pattern. It is repeated/fixed within every trial.


### Import library

In [1]:

import random
import os
from os import listdir
import pandas as pd
from pydub import AudioSegment
import itertools
from pydub.playback import play
import numpy as np


c:\Users\l2016\Anaconda3\envs\LiMN\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


### Import data 

In [2]:

'''
To generate sequences, call the simplified_spreadsheet() function. An example is included
below the function.
shifts - 3 semitone apart; the number is the number of semitones shifted
(ex: ba-changesemitones_12.wav)
duration cut to 0.41756 seconds each

OK things to do
- balance where the change occurs in the sequence!

'''
# stim_dir = "words-pitch-shifted-mono/"
# save_dir = 'C:/Users/elibu/PycharmProjects/AVPOT/auditory-stimuli-mono/'

#word stim with 3 semitone dif
#stim_dir = "E:/cmu/lab project/words-monotonized-pitch-shift/"
#word stim with 5 semitone dif
#stim_dir = "E:/cmu/lab project/words-monotonized-5-semitone-shifts/"

#syllabes stim with 3 semitone dif
#stim_dir = "E:/cmu/lab project/stimuli_sets_new/Monotonized 3 semitone shifts/"
#syllabes stim with 4 semitone dif
stim_dir = "E:/cmu/lab project/stimuli_sets_new/Monotonized 4 semitone shifts/"
#syllabes stim with 5 semitone dif
#stim_dir = "E:/cmu/lab project/stimuli_sets_new/Monotonized 5 semitone shifts/"


# female1 = [AudioSegment.from_wav(stim_dir + f) for f in sorted(listdir(stim_dir))
#            if 'AI' in f]
# female1_names = [f for f in sorted(listdir(stim_dir))
#                  if 'AI' in f]

# male1 = [AudioSegment.from_wav(stim_dir + f) for f in sorted(listdir(stim_dir))
#          if 'AP' in f]
# male1_names = [f for f in sorted(listdir(stim_dir))
#                if 'AP' in f]
#print(female1_names[0:17])


#include the only word
#the sequence is in this order -12 -3 -6 -9 12 3 6 9 0 ; the sorted list and unsorted list are the same
female1 = [AudioSegment.from_wav(stim_dir + f) for f in listdir(stim_dir)
           if 'AI' and '200' in f]
female1_names = [f for f in listdir(stim_dir)
                 if 'AI' and '200' in f]

male1 = [AudioSegment.from_wav(stim_dir + f) for f in listdir(stim_dir)
         if 'AP' and '100' in f]
male1_names = [f for f in sorted(listdir(stim_dir))
               if 'AP' and '100' in f]
# print(len(female1_names))
# print(len(male1_names))
# print(female1_names[0:17])
# print(male1_names[0:17])


all_speakers = ["AI", "AP"]
#word
#all_syllables = ["balm", "bean", "boon", "dawn", "deem", "dual", "gall", "gear", "goose"]
#syllables
all_syllables = ["ba", "bee", "boo", "da", "dee", "doo", "ga", "gee", "goo"]

# all_speakers_perms = [list(zip(each_permutation, all_speakers)) for each_permutation in itertools.permutations(all_syllables, len(all_speakers))]
speaker_wavs = female1 + male1
speaker_wavs_names = female1_names + male1_names

male1_matrix = np.zeros((9, 9), dtype=object)
female1_matrix = np.zeros((9, 9), dtype=object)
male1_matrix_names = np.zeros((9, 9), dtype=object)
female1_matrix_names = np.zeros((9, 9), dtype=object)

# 9x9 matrix - column j for pitch, row i for word
for i in range(9):
    for j in range(9):
        male1_matrix[i, j] = male1[9 * i + j]
        female1_matrix[i, j] = female1[9 * i + j]
        male1_matrix_names[i, j] = male1_names[9 * i + j]
        female1_matrix_names[i, j] = female1_names[9 * i + j]

### functions

In [3]:
def gen_trial_from_filenames(seq1, seq2, iItemi, iSeqi, seq_length):
    """
    :param seq1:        sound sequence 1 
    :param seq2:        sound sequence 2
    :param iItemi:      the pause between every item in the sequence
    :param iSeqi:       the pause between each sequence
    :param seq_length:  the length of the sequence 1, 2
    :return:            the trial sequence 
    """

    from os import listdir
    import pandas as pd
    from pydub import AudioSegment
    import itertools
    from pydub.playback import play

    pause_sound = AudioSegment.silent(duration=iItemi)

    pause_btw_seq_sound = AudioSegment.silent(duration=iSeqi)

    sequence_1 = seq1[0]
    sequence_2 = seq2[0]

    for i in range(1, seq_length):
        sequence_1 = sequence_1 + pause_sound + seq1[i]

    for i in range(1, len(seq2)):
        sequence_2 = sequence_2 + pause_sound + seq2[i]

    trial = sequence_1 + pause_btw_seq_sound + sequence_2

    return trial


In [4]:
# Change the first sequence to
# The sequence length have to be smaller than 9, otherwise np.setdiff1d will return empty array
def generate_sequence(dir, file_matrix, seq_length, trial_type, pause_item, pause_seq,
                      attend_cond, filename_matrix, name, serial):
    #define the paramter 
    """
    :param dir:             the saving wave file directory
    :param file_matrix:     the matrix with (Row(words) by Column(pitches))
    :param seq_length:      length of the sequence generated 
    :param trial_type:      one of 4 cases same/different(word/pitch) present/abscent(word/pitch)
    :param pause_item:      the pause between every item in the sequence
    :param pause_seq:       the pause between each sequence
    :param attend_cond:     attend to the "pitch" or "word"
    :param filename_matrix: the matrix with names of (Row(words) by Column(pitches))
    :param name:            name the sequence after the trail type
    :param serial:          target index 0-8 (1-9) 
    :return:
    """
    import numpy as np

    # Construct the 1st sequence
    if attend_cond == 'pitch':
        # 1st sequence: randomize the pitch & keep word same
        pitch_list = np.random.choice(np.arange(9), size=seq_length, replace=False)
        word_list = np.full(seq_length, np.random.randint(0, 9))
                    
    elif attend_cond == 'word':
        # 1st sequence: randomize the word & keep pitch same
        word_list = np.random.choice(np.arange(9), size=seq_length, replace=False)
        pitch_list = np.full(seq_length, np.random.randint(0, 9))

    else:
        raise Exception('Attend condition inputted is not valid')
        
    # list of tuples containing index pair corresponding to pitch/word instance
    items = [(word_list[i], pitch_list[i]) for i in range(0, seq_length)]
    
    seq_1 = np.zeros((seq_length,), dtype=object)
    seq_1_names = np.zeros((seq_length,), dtype=object)
    for i in range(seq_length):
        seq_1[i] = file_matrix[items[i]]
        seq_1_names[i] = filename_matrix[items[i]] 


    if trial_type == 'same':
        # 2nd sequence: Non randomized second dimension
        seq_2 = seq_1.copy() 
        seq_2_names = seq_1_names 

    elif trial_type == 'different': 
        #Initiation the sequence & switch position
        switch_pos = serial
        seq_2 = np.zeros((seq_length,), dtype=object)
        seq_2_names = np.zeros((seq_length,), dtype=object)
        
        if attend_cond == 'pitch':
            #print('pitch')
            # swap adjacent pitch indices in pitch list
            pitch_list_diff = pitch_list.copy()
            pitch_list_diff[switch_pos], pitch_list_diff[switch_pos + 1] = \
                pitch_list_diff[switch_pos + 1], pitch_list_diff[switch_pos]
            #Unchanged in word_list
            items_seq_diff = [(word_list[i], pitch_list_diff[i]) for i in range(0, seq_length)]

        elif attend_cond == 'word':
            #print('word')
            # swap adjacent word indices
            word_list_diff = word_list.copy()
            word_list_diff[switch_pos], word_list_diff[switch_pos + 1] = \
                word_list_diff[switch_pos + 1], word_list_diff[switch_pos]
            #keep the pitches same
            items_seq_diff = [(word_list_diff[i], pitch_list[i]) for i in range(0, seq_length)]

        else:
            raise Exception('Attend condition inputted is not valid')

        #Assignt the value for 2nd sequence    
        for i in range(seq_length):
            seq_2[i] = file_matrix[items_seq_diff[i]]
            seq_2_names[i] = filename_matrix[items_seq_diff[i]]

        #sanity check
        #print('seq1 names', seq_1_names)
        #print('seq2 names', seq_2_names)

    elif trial_type == 'present':
        switch_pos = serial
        #randomized dimension -- depends on the attending dimension
        if attend_cond == 'pitch' or attend_cond == 'word':
            seq_2 = [seq_1[switch_pos]]
            seq_2_names = seq_1_names[switch_pos]
        else:
            raise Exception("attend condition is inputted incorrectly")

    elif trial_type == 'absent':
        switch_pos = serial
        if attend_cond == 'pitch':  #change the pitch, keep the words the same
            pitch2use = np.random.choice(np.setdiff1d(np.arange(9), pitch_list))
            seq_2 = [file_matrix[(word_list[switch_pos], pitch2use)]]
            seq_2_names = filename_matrix[(word_list[switch_pos], pitch2use)]
        elif attend_cond == 'word': #change the word, keep the pitches the same
            word2use = np.random.choice(np.setdiff1d(np.arange(9), pitch_list))
            seq_2 = [file_matrix[(word2use, pitch_list[switch_pos])]]
            seq_2_names = filename_matrix[(word2use, pitch_list[switch_pos])]
        else:
            raise Exception("attend condition is inputted incorrectly")
       
    else:
        raise Exception("Trial type is incorrect")

    # call function that creates the sequence using pydub
    trial = gen_trial_from_filenames(seq1=seq_1, seq2=seq_2,
                                     iItemi=pause_item,
                                     iSeqi=pause_seq,
                                     seq_length=seq_length)

    trial.export(dir + name, format="wav")

    return trial, seq_1_names, seq_2_names, switch_pos

In [5]:
def simplified_spreadsheet_word_pitches(block_num, task_type, num, audio, audio_names, name_start, length, pause, pause_seq_len,
                                        aud_type, save_dir_str):
    """
    Generate same/different or absent/present sequences as wav files.
    :param block_num:       the block number generated: ex 1, 2, 3, 4 ...
    :param task_type:       'sd' for same/different or 'ap' for absent/present
    :param num:             number of sequences to generate for absent/present or same/different Question: num of trails per condition (check)
    :param audio:           list of all audio segments to generate sequences from
    :param audio_names:     names of all audio segments to generate sequences from, in the same order as audio
    :param name_start:      start of what you want to name each sequence
    :param length:          number of audio segments in each sequence
    :param pause:           ms of silence between each audio segment in the sequence
    :param pause_seq_len:   ms of silence between sequences
    :param aud_type:        "single" or "all" for a single speaker or all speakers  #this is metadata single right now
    :param save_dir:        the saving directory for the .wav and excel files
    :return:                num files are saved to the file path in stimula_data.to_excel()
    """
    attend_conditions = ['word', 'pitch']

    for att_cond in attend_conditions:
        # initialize dataframe to hold parameters and results
        stimuli_data = pd.DataFrame({'randomise_blocks': [None],
                                        'randomise_trials': [None],
                                        'display': [None],
                                        'Audio_Filename': [None],
                                        'Same_Different': [None],
                                        'Absent_Present': [None], #added new column
                                        'text': [None],
                                        'embedded':[None], #added new column
                                        'Attend_Condition': [None],
                                        'Length': [None],
                                        'Pause': [None],
                                        'Sequence Pause': [None],
                                        'Sequence_Names': [None],   #1st sequence name
                                        'Switch_Names': [None],     #2nd sequence name
                                        'Stim_Ind' :[None]  # added new column
                                        #'Stim_Name' :[None]
                                    })

        if task_type == 'sd': #Same Different task
            #change save dir based on task 
            save_dir = save_dir_str + 'task_type_' + task_type +'/Block_' + str(block_num) + '_' + att_cond + '/len' + str(length) + '/'
            if not os.path.isdir(save_dir):
                os.makedirs(save_dir)

            for i in range(num):
                # generate num calls to generate_audio; each call generates a same and different
                #   audio file
                for j in range(0, length - 1):
                    same_name = f'Block_{block_num}_' + name_start + f'_same_{att_cond}_{length}_serial_{j}_stim' + str(i+1) + '.wav'
                    diff_name = f'Block_{block_num}_' + name_start + f'_diff_{att_cond}_{length}_serial_{j}_stim' + str(i+1) + '.wav'

                    same, first_seq_names, second_seq_names, switch_inds_same = generate_sequence(dir = save_dir,
                                                                                                    file_matrix=audio,
                                                                                                    seq_length=length,
                                                                                                    trial_type='same',
                                                                                                    pause_item=pause,
                                                                                                    pause_seq=pause_seq_len,
                                                                                                    attend_cond=att_cond,
                                                                                                    filename_matrix=audio_names,
                                                                                                    name=same_name,
                                                                                                    serial=j)

                    diff, first_seq_names_diff, second_seq_names_diff, switch_inds_diff = generate_sequence(dir = save_dir,
                                                                                                    file_matrix=audio,
                                                                                                    seq_length=length,
                                                                                                    trial_type='different',
                                                                                                    pause_item=pause,
                                                                                                    pause_seq=pause_seq_len,
                                                                                                    attend_cond=att_cond,
                                                                                                    filename_matrix=audio_names,
                                                                                                    name=diff_name,
                                                                                                    serial=j)

                    # create temporary dataframe for each audio file with parameters
                    #same task
                    temp_df_1 = pd.DataFrame({'randomise_blocks': [None],
                                              'randomise_trials': 1,
                                              'display': 'task-sd',
                                              'Audio_Filename': same_name,
                                              'Same_Different': 'same',
                                              'Absent_Present':[None],
                                              'text': [None],
                                              'embedded':[None],
                                              'Attend_Condition': '### '+ att_cond,
                                              'Length': length,
                                              'Pause': pause,
                                              'Sequence Pause': pause_seq_len,
                                              'Sequence_Names': str(first_seq_names),
                                              #'Second_Sequence_Names': str(second_seq_names),
                                              'Switch_Names':  str(second_seq_names),
                                              'Stim_Ind' : [None]
                                              }, index=[0])

                    #different task
                    temp_df_2 = pd.DataFrame({'randomise_blocks': [None],
                                              'randomise_trials': 1,
                                              'display': 'task-sd',
                                              'Audio_Filename': diff_name,
                                              'Same_Different': 'different',
                                              'Absent_Present':[None],
                                              'text': [None],
                                              'embedded':[None],
                                              'Attend_Condition': '### '+ att_cond,
                                              'Length': length,
                                              'Pause': pause,
                                              'Sequence Pause': pause_seq_len,
                                              'Sequence_Names': str(first_seq_names_diff),
                                              #'Second_Sequence_Names': str(second_seq_names_diff),
                                              'Switch_Names': str(second_seq_names_diff),
                                              'Stim_Ind' : switch_inds_diff
                                              }, index=[0])

                    # append to participant dataframe
                    stimuli_data = pd.concat([stimuli_data, temp_df_1], ignore_index=True)
                    stimuli_data = pd.concat([stimuli_data, temp_df_2], ignore_index=True)

            # convert to csv file           
            stimuli_data.to_excel(
                save_dir +'Block_' + str(block_num) +'_' + task_type + '_' + name_start + '_' + att_cond + '_len' + str(length) + '.xlsx',
                index=False)
        
        elif task_type == 'ap':
            #change save dir based on task 
            save_dir = save_dir_str + 'task_type_' + task_type +'/Block_' + str(block_num) + '_' + att_cond + '/len' + str(length) + '/'
            if not os.path.isdir(save_dir):
                os.makedirs(save_dir)

            # # initialize dataframe to hold parameters and results
            # stimuli_data = pd.DataFrame({'randomise_blocks': [None],
            #                              'randomise_trials': [None],
            #                              'display': [None],
            #                              'Audio_Filename': [None],
            #                              'Absent_Present': [None],
            #                              'text': [None],
            #                              'Length': [None],
            #                              'Pause': [None],
            #                              'Sequence Pause': [None],
            #                              'Sequence_Names': [None],
            #                              'Stim_Name': [None],
            #                              'Stim_Ind': [None],
            #                              'Attend_Condition': [None]
            #                              })

            for i in range(num):
                for j in range(0, length):
                    absent_name = f'Block_{block_num}_' + name_start + f'_absent_{att_cond}_{length}_serial_{j}_' + str(i+1) + '.wav'
                    present_name = f'Block_{block_num}_' + name_start + f'_present_{att_cond}_{length}_serial_{j}_' + str(i+1) + '.wav'

                    absent, seq_names_absent, absent_stim_name, switch_inds_absent = generate_sequence(dir = save_dir,
                                                                                                        file_matrix=audio,
                                                                                                        seq_length=length,
                                                                                                        trial_type='absent',
                                                                                                        pause_item=pause,
                                                                                                        pause_seq=pause_seq_len,
                                                                                                        attend_cond=att_cond,
                                                                                                        filename_matrix=audio_names,
                                                                                                        name=absent_name,
                                                                                                        serial=j)

                    present, seq_names_present, present_stim_name, present_stim_ind = generate_sequence(dir = save_dir,
                                                                                                        file_matrix=audio,
                                                                                                        seq_length=length,
                                                                                                        trial_type='present',
                                                                                                        pause_item=pause,
                                                                                                        pause_seq=pause_seq_len,
                                                                                                        attend_cond=att_cond,
                                                                                                        filename_matrix=audio_names,
                                                                                                        name=present_name,
                                                                                                        serial=j)

                    temp_df_1 = pd.DataFrame({'randomise_blocks': [None],
                                              'randomise_trials': 1,
                                              'display': 'task-ap',
                                              'Audio_Filename': absent_name,
                                              'Same_Different':[None],
                                              'Absent_Present': 'absent',
                                              'text': [None],
                                              'embedded':[None],
                                              'Attend_Condition':'### '+ att_cond,
                                              'Length': length,
                                              'Pause': pause,
                                              'Sequence Pause': pause_seq_len,
                                              'Sequence_Names': str(seq_names_absent),
                                              'Switch_Names': absent_stim_name,
                                              'Stim_Ind': j
                                              }, index=[0])

                    temp_df_2 = pd.DataFrame({'randomise_blocks': [None],
                                              'randomise_trials': 1,
                                              'display': 'task-ap',
                                              'Audio_Filename': present_name,
                                              'Same_Different':[None],
                                              'Absent_Present': 'present',
                                              'text': [None],
                                              'embedded':[None],
                                              'Attend_Condition':'### '+ att_cond,
                                              'Length': length,
                                              'Pause': pause,
                                              'Sequence Pause': pause_seq_len,
                                              'Sequence_Names': str(seq_names_present),
                                              'Switch_Names': present_stim_name,
                                              'Stim_Ind': j
                                              }, index=[0])

                    # append to participant dataframe
                    stimuli_data = pd.concat([stimuli_data, temp_df_1], ignore_index=True)
                    stimuli_data = pd.concat([stimuli_data, temp_df_2], ignore_index=True)

            # convert to csv file           
            stimuli_data.to_excel(
                save_dir +'Block_' + str(block_num) +'_' + task_type + '_' + name_start + '_' + att_cond + '_len' + str(length) + '.xlsx',
                index=False)
                
        else:
            raise Exception("Task type is incorrect")

    return

### generate sequence

In [6]:
n_stim_per_cond = 1

# save_dir_v4 = "E:/cmu/lab project/stimuli generation/stimuli_v4/"
# #PLEASE delete the existing directory called save_dir_v2 before running the code
# if not os.path.isdir(save_dir_v4):
#     os.makedirs(save_dir_v4)

#Generate num_block blocks for num_length start at length 4(included) 
sequenceInterval = 2000
num_block = 1

length = [6]
interval = [10]

for i in interval:
    save_dir_v3 = "E:/cmu/lab project/stimuli generation/V4-syllable-4-semitone-interval-" + str(i)+"/"
    for block in range (1,1+num_block):
        print('block number:', block)
        for l in length:
            print('length:', l)
            #P/A task male and female
            #simplified_spreadsheet_word_pitches(block_num, task_type, num, audio, audio_names, name_start, length, pause, pause_seq_len, aud_type, save_dir_str)
            simplified_spreadsheet_word_pitches(block,'ap', n_stim_per_cond, male1_matrix, male1_matrix_names, 'male_1speaker', l, i, sequenceInterval, "single", save_dir_v3)
            simplified_spreadsheet_word_pitches(block,'ap', n_stim_per_cond, female1_matrix, female1_matrix_names, 'female_1speaker', l, i, sequenceInterval, "single", save_dir_v3)

            #S/D task male and female
            #simplified_spreadsheet_word_pitches(block_num, task_type, num, audio, audio_names, name_start, length, pause, pause_seq_len, aud_type, save_dir_str)
            simplified_spreadsheet_word_pitches(block,'sd', n_stim_per_cond, male1_matrix, male1_matrix_names, 'male_1speaker', l, i, sequenceInterval, "single", save_dir_v3)
            simplified_spreadsheet_word_pitches(block,'sd', n_stim_per_cond, female1_matrix, female1_matrix_names, 'female_1speaker', l, i, sequenceInterval, "single", save_dir_v3)

In [1]:
length = [6,8,10]
for l in length:
    print(l)

6
8
10
